In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
                 names=['user_id', 'item_id', 'rating', 'timestamp']) 
 
r = df.pivot(index='user_id', columns='item_id', values='rating').values

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
def ls_recommender(r, alpha=0.001) -> np.ndarray:
  beta_user = np.random.random(len(r))
  beta_item = np.random.random(len(r[0]))
  not_nan_indices = np.argwhere(np.logical_not(np.isnan(r)))

  print("starting sgd")
  y_pred = np.ones(r.shape) * np.nan

  for iteration in range(1000):
    for index in not_nan_indices:
      y_pred[index[0]][index[1]] = beta_user[index[0]] + beta_item[index[1]]

    g_b0 = -1 * np.nansum(np.dstack((r,-y_pred)),2)
    g_b1 = -1 * np.nansum(np.dstack((r,-y_pred)),2)

    #print(f"({i}) beta_user: {beta_user}, beta_item: {beta_item}, gradient: {g_b0} {g_b1}")

    beta_prev_user = np.copy(beta_user)
    beta_prev_item = np.copy(beta_item)

    for i,u_beta in enumerate(beta_user):
      beta_user[i] = u_beta - (np.nansum(g_b0[i])*alpha)

    for j,i_beta in enumerate(beta_item):
      beta_item[j] = i_beta - (np.nansum(g_b1[:,j])*alpha)
          

    if np.linalg.norm(beta_user - beta_prev_user) < 0.0001 and np.linalg.norm(beta_item - beta_prev_item) < 0.0001:
        print(f"I do early stoping at iteration {iteration}")
        break

  return beta_user, beta_item

In [4]:
def ls_recommender_modified(r, alpha=0.001, hyperparameter_lambda = 0.01) -> np.ndarray:
  beta_user = np.random.random(len(r))
  beta_item = np.random.random(len(r[0]))
  not_nan_indices = np.argwhere(np.logical_not(np.isnan(r)))

  print("starting sgd")
  y_pred = np.ones(r.shape) * np.nan

  for iteration in range(100):
    for index in not_nan_indices:
      y_pred[index[0]][index[1]] = beta_user[index[0]] + beta_item[index[1]]

    g_b_user = (-1 * np.nansum(np.dstack((r,-y_pred)),2)) + (hyperparameter_lambda * np.nansum(beta_user))
    g_b_item = (-1 * np.nansum(np.dstack((r,-y_pred)),2)) + (hyperparameter_lambda * np.nansum(beta_item))

    #print(f"({i}) beta_user: {beta_user}, beta_item: {beta_item}, gradient: {g_b0} {g_b1}")

    beta_prev_user = np.copy(beta_user)
    beta_prev_item = np.copy(beta_item)

    for i,u_beta in enumerate(beta_user):
      beta_user[i] = u_beta - (np.nansum(g_b_user[i])*alpha)

    for j,i_beta in enumerate(beta_item):
      beta_item[j] = i_beta - (np.nansum(g_b_item[:,j])*alpha)
          

    if np.linalg.norm(np.nansum(beta_user - beta_prev_user)) < 0.01 and np.linalg.norm(np.nansum(beta_item - beta_prev_item)) < 0.01:
        print(f"I do early stoping at iteration {iteration}")
        break

  return beta_user, beta_item

# **PART 1**

In [6]:
beta_user, beta_item = ls_recommender(r, alpha=0.001)

starting sgd


In [7]:
def calc_err_modified(beta_user, beta_item, r):
  residual_sum_of_squares = 0
  not_nan_indices = np.argwhere(np.logical_not(np.isnan(r)))
  for index in not_nan_indices:
    i = index[0]
    j = index[1]
    y_hat = beta_user[i] + beta_item[j]
    y = r[i][j]
    residual_sum_of_squares += (y_hat - y) ** 2
  error = residual_sum_of_squares/2
  return error

In [8]:
calc_err_modified(beta_user, beta_item, r)

41717.92865046914

# **PART 2**

In [9]:
beta_user_modified, beta_item_modified = ls_recommender_modified(r, alpha=0.001, hyperparameter_lambda = 0.01)

starting sgd


In [12]:
def calc_err_modified(beta_user, beta_item, r, hyperparameter_lambda = 0.01):
  residual_sum_of_squares = 0
  not_nan_indices = np.argwhere(np.logical_not(np.isnan(r)))
  for index in not_nan_indices:
    i = index[0]
    j = index[1]
    y_hat = beta_user[i] + beta_item[j]
    y = r[i][j]
    residual_sum_of_squares += (y_hat - y) ** 2
    error_first_part = residual_sum_of_squares/2
    error_modified_part = (hyperparameter_lambda * (np.nansum(np.power(beta_user,2)) + np.nansum(np.power(beta_item,2)))) / 2 
    error = error_first_part + error_modified_part
  return error


In [13]:
calc_err_modified(beta_user_modified, beta_item_modified, r, 0.01)

1.1716775550052553e+240